In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
import glob
from google.cloud import bigquery
import re
import openpyxl 
import re
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
client = bigquery.Client()

In [3]:

query = ''' SELECT 

 cbo_2002,


  CASE 
    WHEN natureza_juridica IN ('1015','1040', '1074', '1104','1139', '1163', '1252', '1287', '1317', '1341') THEN 'Federal'
    WHEN natureza_juridica IN ('1023','1058', '1082', '1112', '1147', '1171', '1236','1260',  '1295', '1325') THEN 'Estadual'
    WHEN natureza_juridica IN ('1031','1066', '1120', '1155', '1180', '1244', '1279', '1309', '1333') THEN 'Municipal'
    ELSE 'Outros' 
  END AS esfera,

   CASE 
    WHEN sexo = '1' THEN 'Masculino'
    WHEN sexo = '2' THEN 'Feminino'
    ELSE 'Não identificado'
  END AS sexo,



  COUNT(*) AS quantidade_vinculos
  
  --avg(valor_remuneracao_media)as media


FROM `basedosdados.br_me_rais.microdados_vinculos`
WHERE (natureza_juridica LIKE "1%" 
OR natureza_juridica IN ('2011', '2038'))
AND natureza_juridica != '1228'
AND cbo_2002 NOT LIKE "0%" -- retirando os militares. 
AND vinculo_ativo_3112= '1' and ano=2003

GROUP BY 

cbo_2002,esfera,sexo

order by quantidade_vinculos desc
-- Total de 2105 codigos


--Quantidade de vinculos:
-- cbo, familia cbo, genero, esfera, poder

--Remuneração por ocupação:
-- cbo, familia cbo, genero, esfera, poder, tipo de adm (empresa pública devem ser as maiores)



 
 --Assistente administrativo
 --Professor de nível médio no ensino fundamental
 -- Professor de nível superior do ensino fundamental (primeira a quarta série)
 -- Dirigente do serviço público municipal
 -- Auxiliar de escritório
  '''

In [4]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df


Downloading: 100%|██████████|


,cbo_2002,esfera,sexo,quantidade_vinculos
0,331205,Estadual,Feminino,297234
1,331205,Municipal,Feminino,226829
2,231210,Municipal,Feminino,199570
3,411010,Estadual,Feminino,189629
4,411005,Estadual,Feminino,187276
...,...,...,...,...
8801,262605,Federal,Masculino,1
8802,848325,Federal,Masculino,1
8803,373220,Outros,Feminino,1
8804,211110,Estadual,Feminino,1


In [5]:
df

,cbo_2002,esfera,sexo,quantidade_vinculos
0,331205,Estadual,Feminino,297234
1,331205,Municipal,Feminino,226829
2,231210,Municipal,Feminino,199570
3,411010,Estadual,Feminino,189629
4,411005,Estadual,Feminino,187276
...,...,...,...,...
8801,524210,Federal,Masculino,1
8802,261820,Outros,Feminino,1
8803,761354,Outros,Masculino,1
8804,318205,Municipal,Masculino,1


In [6]:
df = df.rename(columns={'cbo_2002':'codigo'})
df

,codigo,esfera,sexo,quantidade_vinculos
0,331205,Estadual,Feminino,297234
1,331205,Municipal,Feminino,226829
2,231210,Municipal,Feminino,199570
3,411010,Estadual,Feminino,189629
4,411005,Estadual,Feminino,187276
...,...,...,...,...
8801,524210,Federal,Masculino,1
8802,261820,Outros,Feminino,1
8803,761354,Outros,Masculino,1
8804,318205,Municipal,Masculino,1


In [7]:
df.dtypes

codigo                 object
esfera                 object
sexo                   object
quantidade_vinculos     Int64
dtype: object

In [8]:
df['codigo'] = df['codigo'].astype(int)

Pegar dados cbo e dar merge com codigo do arquivo > CBO2002 - Ocupacao
subir pro datalake
desenhar gráfico


In [16]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CBO\\ESTRUTURA CBO')

In [17]:
os.listdir() #ocupação, familia, sub grupo, grupo principal, 

['CBO2002 - Sinonimo.csv',
 'CBO2002 - PerfilOcupacional.csv',
 'CBO2002 - SubGrupo Principal.csv',
 'CBO2002 - Grande Grupo.csv',
 'CBO2002 - Familia.csv',
 'CBO2002 - SubGrupo.csv',
 'titulo_funcoes_cbo.xlsx',
 'CBO2002 - SubGrupo Principal.gsheet',
 'CBO2002 - SubGrupo.gsheet',
 'comparando dados CBO.gsheet',
 'CBO2002 - Grande Grupo.gsheet',
 'cbo atualizado.xlsx',
 'cbo_2003.csv',
 'cbo_quadro.xlsx',
 'CBO2002 - Ocupação (1).xlsx']

In [18]:
cbo = pd.read_excel("CBO2002 - Ocupação (1).xlsx")
cbo

,CODIGO,TITULO,Agrupamento
0,10105,Oficial general da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
1,10110,Oficial general do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
2,10115,Oficial general da marinha,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
3,10205,Oficial da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
4,10210,Oficial do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2709,992210,Encarregado de equipe de conservação de vias p...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2710,992215,Operador de ceifadeira na conservação de vias ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2711,992220,Pedreiro de conservação de vias permanentes (e...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...


In [19]:
cbo[cbo['TITULO']=='Agente de saúde pública']

,CODIGO,TITULO,Agrupamento
1141,352210,Agente de saúde pública,"PROFISSIONAIS DE MEDICINA, ENFERMAGEM E SERVIÇ..."


In [20]:
cbo['Agrupamento'].unique()

array(['MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEIROS MILITARES',
       'MEMBROS SUPERIORES DO PODER PÚBLICO, DIRIGENTES DE ORGANIZAÇÕES DE INTERESSE PÚBLICO E DE EMPRESAS, GERENTES',
       'PROFISSIONAIS DE MEDICINA, ENFERMAGEM E SERVIÇOS DA SAÚDE',
       'PROFISSIONAIS DE CIÊNCIA DE ENSINO SUPERIOR',
       'PROFISSIONAIS DE ENSINO',
       'PROFISSIONAIS DE ARTES DE ENSINO SUPERIOR',
       'TÉCNICOS DE NÍVEL MÉDIO',
       'TRABALHADORES DE SERVIÇOS ADMINISTRATIVOS',
       'TRABALHADORES DOS SERVIÇOS, VENDEDORES DO COMÉRCIO EM LOJAS E MERCADOS',
       'TRABALHADORES NOS SERVIÇOS DE PROTEÇÃO E SEGURANÇA',
       'TRABALHADORES AGROPECUÁRIOS, FLORESTAIS E DA PESCA',
       'TRABALHADORES DA PRODUÇÃO DE BENS E SERVIÇOS INDUSTRIAIS',
       'TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUTENÇÃO'],
      dtype=object)

In [21]:
cbo = cbo.rename(columns=
                 {
                     'CODIGO':'codigo',
                     'TITULO':'nome_ocupacao',
                     'Agrupamento':'grupo_ocupacao'
                 })
cbo

,codigo,nome_ocupacao,grupo_ocupacao
0,10105,Oficial general da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
1,10110,Oficial general do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
2,10115,Oficial general da marinha,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
3,10205,Oficial da aeronáutica,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
4,10210,Oficial do exército,"MEMBROS DAS FORÇAS ARMADAS, POLICIAIS E BOMBEI..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2709,992210,Encarregado de equipe de conservação de vias p...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2710,992215,Operador de ceifadeira na conservação de vias ...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...
2711,992220,Pedreiro de conservação de vias permanentes (e...,TRABALHADORES EM SERVIÇOS DE REPARAÇÃO E MANUT...


In [22]:
cbo[cbo['codigo']==517235]

,codigo,nome_ocupacao,grupo_ocupacao
1513,517235,Guarda portuário,TRABALHADORES NOS SERVIÇOS DE PROTEÇÃO E SEGUR...


In [23]:
cbo['grupo_ocupacao'] = cbo['grupo_ocupacao'].str.title()
cbo

,codigo,nome_ocupacao,grupo_ocupacao
0,10105,Oficial general da aeronáutica,"Membros Das Forças Armadas, Policiais E Bombei..."
1,10110,Oficial general do exército,"Membros Das Forças Armadas, Policiais E Bombei..."
2,10115,Oficial general da marinha,"Membros Das Forças Armadas, Policiais E Bombei..."
3,10205,Oficial da aeronáutica,"Membros Das Forças Armadas, Policiais E Bombei..."
4,10210,Oficial do exército,"Membros Das Forças Armadas, Policiais E Bombei..."
...,...,...,...
2708,992205,Encarregado geral de operações de conservação ...,Trabalhadores Em Serviços De Reparação E Manut...
2709,992210,Encarregado de equipe de conservação de vias p...,Trabalhadores Em Serviços De Reparação E Manut...
2710,992215,Operador de ceifadeira na conservação de vias ...,Trabalhadores Em Serviços De Reparação E Manut...
2711,992220,Pedreiro de conservação de vias permanentes (e...,Trabalhadores Em Serviços De Reparação E Manut...


In [24]:
cbo_merge = df.merge(cbo, how='left', on='codigo') #Merge RAIS e CBO
cbo_merge  

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,331205,Estadual,Feminino,297234,Professor de nível médio no ensino fundamental,Profissionais De Ensino
1,331205,Municipal,Feminino,226829,Professor de nível médio no ensino fundamental,Profissionais De Ensino
2,231210,Municipal,Feminino,199570,Professor de nível superior do ensino fundamen...,Profissionais De Ensino
3,411010,Estadual,Feminino,189629,Assistente administrativo,Trabalhadores De Serviços Administrativos
4,411005,Estadual,Feminino,187276,Auxiliar de escritório,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (em banca de jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé..."
8802,261820,Outros,Feminino,1,Repórter fotográfico,Profissionais De Ciência De Ensino Superior
8803,761354,Outros,Masculino,1,Operador de máquina de cordoalha,Trabalhadores Da Produção De Bens E Serviços I...
8804,318205,Municipal,Masculino,1,Desenhista técnico mecânico,Técnicos De Nível Médio


In [25]:
cbo_merge.dtypes

codigo                  int32
esfera                 object
sexo                   object
quantidade_vinculos     Int64
nome_ocupacao          object
grupo_ocupacao         object
dtype: object

In [26]:
cbo_merge['grupo_ocupacao'].unique()

array(['Profissionais De Ensino',
       'Trabalhadores De Serviços Administrativos',
       'Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       nan,
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Técnicos De Nível Médio',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Profissionais De Ciência De Ensino Superior',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Profissionais De Artes De Ensino Superior'], dtype=object)

In [27]:
len(cbo_merge['grupo_ocupacao'].unique()) # 13 categorias com NaN

13

In [28]:
cod_obsoleto = cbo_merge[cbo_merge['nome_ocupacao'].isna()] #codigos excluidos e desatualizados
cod_obsoleto 

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
11,514210,Municipal,Feminino,94793,NaN,NaN
23,991405,Municipal,Feminino,60461,NaN,NaN
39,991405,Municipal,Masculino,35175,NaN,NaN
43,223115,Municipal,Masculino,33942,NaN,NaN
58,223115,Estadual,Masculino,25533,NaN,NaN
...,...,...,...,...,...,...
8736,223620,Outros,Masculino,1,NaN,NaN
8752,373120,Outros,Feminino,1,NaN,NaN
8761,223139,Federal,Masculino,1,NaN,NaN
8767,991410,Estadual,Feminino,1,NaN,NaN


In [29]:
len(cod_obsoleto)

452

In [30]:
cod_obsoleto['codigo'].unique()

array([514210, 991405, 223115, 517315, 223149, 223110, 223610, 223132,
       223410, 223137, 223620, 342520, 510125, 223116, 223146, 223156,
       223105, 253105, 223150, 223129, 223104, 514220, 223106, 991410,
       324210, 223153, 991415, 223118, 373205, 223124, 422215, 223144,
       322105, 223117, 223142, 223147, 223157, 223111, 223102, 223140,
       223119, 223107, 516135, 223122, 223112, 223101, 223151, 223136,
       223133, 223103, 373125, 373110, 223125, 223128, 223126, 223134,
       223145, 223135, 223148, 373215, 223120, 223154, 373120, 373210,
       223113, 223139, 223108, 373115, 223152, 516115, 261720, 354110,
       223615, 223131, 223114, 314605, 722225, 223155, 223138, 223143,
       214355, 352415, 223109, 223127, 223130, 322310, 322115, 223141,
       261725, 223121, 223123])

In [31]:
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,331205,Estadual,Feminino,297234,Professor de nível médio no ensino fundamental,Profissionais De Ensino
1,331205,Municipal,Feminino,226829,Professor de nível médio no ensino fundamental,Profissionais De Ensino
2,231210,Municipal,Feminino,199570,Professor de nível superior do ensino fundamen...,Profissionais De Ensino
3,411010,Estadual,Feminino,189629,Assistente administrativo,Trabalhadores De Serviços Administrativos
4,411005,Estadual,Feminino,187276,Auxiliar de escritório,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (em banca de jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé..."
8802,261820,Outros,Feminino,1,Repórter fotográfico,Profissionais De Ciência De Ensino Superior
8803,761354,Outros,Masculino,1,Operador de máquina de cordoalha,Trabalhadores Da Produção De Bens E Serviços I...
8804,318205,Municipal,Masculino,1,Desenhista técnico mecânico,Técnicos De Nível Médio


In [32]:
atualizar_codigo = {
    514210: 514320,
    991405: 514325,
    223115: 225125,
    517315: 517230,
    223149: 225124,
    223110: 225225,
    223610: 223810,
    223132: 225250,
    223410: 223410,
    223137: 225106,
    223620: 223905,
    342520: 141805,
    510125: 271110,
    223116: 225130,
    223146: 225270,
    223156: 223156,
    223105: 225115,
    253105: 142325,
    223150: 223150,
    223129: 225170,
    223104: 225205,
    514220: 514305,
    223106: 225120,
    991410: 991410,
    324210: 324210,
    223153: 225133,
    991415: 514315,
    223118: 225140,
    223124: 225320,
    422215: 422335,
    223144: 225265,
    223117: 225135,
    223142: 225112,
    223147: 225275,
    223157: 225285,
    223111: 225230,
    223102: 225110,
    223140: 225260,
    223119: 223119,
    223107: 225210,
    516135: 322715,
    223122: 225150,
    223112: 225235,
    223101: 225105,
    223151: 223151,
    223136: 225103,
    223133: 225185,
    223103: 225148,
    373125: 373125,
    373110: 373110,
    223125: 225155,
    223128: 225165,
    223126: 225160,
    223134: 225190,
    223145: 225121,
    223135: 225195,
    223148: 225325,
    223120: 225310,
    223154: 225330,
    373120: 373120,
    223113: 225240,
    223139: 225109,
    223108: 225215,
    516115: 322130,
    223152: 225280,
    261720: 261720,
    354110: 253140,
    223615: 223910,
    314605: 314845,
    223131: 225180,
    223114: 225305,
    223155: 225136,
    223143: 223143,
    223138: 225255,
    214355: 202110,
    352415: 142345,
    223127: 225245,
    223109: 225220,
    223130: 225175,
    322310: 322305,
    223141: 225350,
    223121: 225145,
    223123: 225315,
    261725: 261725,
    373205: 373130,
    322105: 322705,
    373215: 373215,
    373210: 373135,
    373115: 373115,
    722225: 722225,
    322115: 322710,
}

In [33]:
cbo_merge['codigo'] = cbo_merge['codigo'].replace(atualizar_codigo) # Feito 


In [34]:
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,331205,Estadual,Feminino,297234,Professor de nível médio no ensino fundamental,Profissionais De Ensino
1,331205,Municipal,Feminino,226829,Professor de nível médio no ensino fundamental,Profissionais De Ensino
2,231210,Municipal,Feminino,199570,Professor de nível superior do ensino fundamen...,Profissionais De Ensino
3,411010,Estadual,Feminino,189629,Assistente administrativo,Trabalhadores De Serviços Administrativos
4,411005,Estadual,Feminino,187276,Auxiliar de escritório,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (em banca de jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé..."
8802,261820,Outros,Feminino,1,Repórter fotográfico,Profissionais De Ciência De Ensino Superior
8803,761354,Outros,Masculino,1,Operador de máquina de cordoalha,Trabalhadores Da Produção De Bens E Serviços I...
8804,318205,Municipal,Masculino,1,Desenhista técnico mecânico,Técnicos De Nível Médio


In [35]:
cod_obsoleto = cbo_merge[cbo_merge['nome_ocupacao'].isna()] # Codigos excluidos e desatualizados
cod_obsoleto 

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
11,514320,Municipal,Feminino,94793,NaN,NaN
23,514325,Municipal,Feminino,60461,NaN,NaN
39,514325,Municipal,Masculino,35175,NaN,NaN
43,225125,Municipal,Masculino,33942,NaN,NaN
58,225125,Estadual,Masculino,25533,NaN,NaN
...,...,...,...,...,...,...
8736,223905,Outros,Masculino,1,NaN,NaN
8752,373120,Outros,Feminino,1,NaN,NaN
8761,225109,Federal,Masculino,1,NaN,NaN
8767,991410,Estadual,Feminino,1,NaN,NaN


In [36]:
df_excel = pd.read_excel('cbo atualizado.xlsx') # Pegando o mapeamento que fiz  
df_excel

,codigo,nome_ocupacao,grupo_ocupacao,ocupação_excluida_flag
0,514320,Faxineiro,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Não
1,514325,Trabalhador da manutenção de edificações,"Trabalhadores Dos Serviços, Vendedores Do Comé...",Não
2,225125,Médico clínico,"Profissionais De Medicina, Enfermagem E Serviç...",Não
3,517230,Policial penal,Trabalhadores Nos Serviços De Proteção E Segur...,Não
4,225124,Médico pediatra,"Profissionais De Medicina, Enfermagem E Serviç...",Não
...,...,...,...,...
86,373215,Ocupação excluída pertencente ao subgrupo Técn...,Técnicos De Nível Médio,NaN
87,373135,Operador de controle mestre,Técnicos De Nível Médio,NaN
88,373115,Ocupação excluída pertencente ao subgrupo Técn...,Técnicos De Nível Médio,NaN
89,722225,Ocupação excluída pertencente ao subgrupo Trab...,Trabalhadores Da Produção De Bens E Serviços I...,NaN


In [37]:
cbo_merge.columns

Index(['codigo', 'esfera', 'sexo', 'quantidade_vinculos', 'nome_ocupacao',
       'grupo_ocupacao'],
      dtype='object')

In [38]:
df_excel.columns

Index(['codigo', 'nome_ocupacao', 'grupo_ocupacao', 'ocupação_excluida_flag'], dtype='object')

In [39]:
temp = pd.merge(cbo_merge, df_excel[['codigo', 'nome_ocupacao', 'grupo_ocupacao']], 
                   on='codigo', how='left')

temp #df temporario para substituir

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao_x,grupo_ocupacao_x,nome_ocupacao_y,grupo_ocupacao_y
0,331205,Estadual,Feminino,297234,Professor de nível médio no ensino fundamental,Profissionais De Ensino,NaN,NaN
1,331205,Municipal,Feminino,226829,Professor de nível médio no ensino fundamental,Profissionais De Ensino,NaN,NaN
2,231210,Municipal,Feminino,199570,Professor de nível superior do ensino fundamen...,Profissionais De Ensino,NaN,NaN
3,411010,Estadual,Feminino,189629,Assistente administrativo,Trabalhadores De Serviços Administrativos,NaN,NaN
4,411005,Estadual,Feminino,187276,Auxiliar de escritório,Trabalhadores De Serviços Administrativos,NaN,NaN
...,...,...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (em banca de jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé...",NaN,NaN
8802,261820,Outros,Feminino,1,Repórter fotográfico,Profissionais De Ciência De Ensino Superior,NaN,NaN
8803,761354,Outros,Masculino,1,Operador de máquina de cordoalha,Trabalhadores Da Produção De Bens E Serviços I...,NaN,NaN
8804,318205,Municipal,Masculino,1,Desenhista técnico mecânico,Técnicos De Nível Médio,NaN,NaN


In [40]:
cbo_merge['nome_ocupacao'] = cbo_merge['nome_ocupacao'].fillna(temp['nome_ocupacao_y'])
cbo_merge['grupo_ocupacao'] = cbo_merge['grupo_ocupacao'].fillna(temp['grupo_ocupacao_y'])

In [41]:
cbo_merge.isna().sum() # Tirei os NaNs

codigo                 0
esfera                 0
sexo                   0
quantidade_vinculos    0
nome_ocupacao          0
grupo_ocupacao         0
dtype: int64

In [42]:
cbo_merge['nome_ocupacao'] = cbo_merge['nome_ocupacao'].str.title()
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,331205,Estadual,Feminino,297234,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
1,331205,Municipal,Feminino,226829,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
2,231210,Municipal,Feminino,199570,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino
3,411010,Estadual,Feminino,189629,Assistente Administrativo,Trabalhadores De Serviços Administrativos
4,411005,Estadual,Feminino,187276,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (Em Banca De Jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé..."
8802,261820,Outros,Feminino,1,Repórter Fotográfico,Profissionais De Ciência De Ensino Superior
8803,761354,Outros,Masculino,1,Operador De Máquina De Cordoalha,Trabalhadores Da Produção De Bens E Serviços I...
8804,318205,Municipal,Masculino,1,Desenhista Técnico Mecânico,Técnicos De Nível Médio


In [43]:
cbo_merge['grupo_ocupacao'] = cbo_merge['grupo_ocupacao'].str.title()
cbo_merge

,codigo,esfera,sexo,quantidade_vinculos,nome_ocupacao,grupo_ocupacao
0,331205,Estadual,Feminino,297234,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
1,331205,Municipal,Feminino,226829,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino
2,231210,Municipal,Feminino,199570,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino
3,411010,Estadual,Feminino,189629,Assistente Administrativo,Trabalhadores De Serviços Administrativos
4,411005,Estadual,Feminino,187276,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos
...,...,...,...,...,...,...
8801,524210,Federal,Masculino,1,Jornaleiro (Em Banca De Jornal),"Trabalhadores Dos Serviços, Vendedores Do Comé..."
8802,261820,Outros,Feminino,1,Repórter Fotográfico,Profissionais De Ciência De Ensino Superior
8803,761354,Outros,Masculino,1,Operador De Máquina De Cordoalha,Trabalhadores Da Produção De Bens E Serviços I...
8804,318205,Municipal,Masculino,1,Desenhista Técnico Mecânico,Técnicos De Nível Médio


In [44]:
cbo_merge = cbo_merge.groupby(['codigo', 'esfera', 'sexo', 'nome_ocupacao', 'grupo_ocupacao'], as_index=False).agg({'quantidade_vinculos': 'sum'}) 
cbo_merge # Agrupar para obter a qtd de vinculos

,codigo,esfera,sexo,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
0,111105,Estadual,Feminino,Senador,"Membros Superiores Do Poder Público, Dirigente...",3
1,111105,Federal,Masculino,Senador,"Membros Superiores Do Poder Público, Dirigente...",2
2,111105,Municipal,Feminino,Senador,"Membros Superiores Do Poder Público, Dirigente...",52
3,111105,Municipal,Masculino,Senador,"Membros Superiores Do Poder Público, Dirigente...",67
4,111110,Estadual,Masculino,Deputado Federal,"Membros Superiores Do Poder Público, Dirigente...",6
...,...,...,...,...,...,...
8801,992225,Federal,Masculino,Auxiliar Geral De Conservação De Vias Permanen...,Trabalhadores Em Serviços De Reparação E Manut...,19
8802,992225,Municipal,Feminino,Auxiliar Geral De Conservação De Vias Permanen...,Trabalhadores Em Serviços De Reparação E Manut...,17078
8803,992225,Municipal,Masculino,Auxiliar Geral De Conservação De Vias Permanen...,Trabalhadores Em Serviços De Reparação E Manut...,20733
8804,992225,Outros,Feminino,Auxiliar Geral De Conservação De Vias Permanen...,Trabalhadores Em Serviços De Reparação E Manut...,365


In [45]:
cbo_merge[cbo_merge['nome_ocupacao']=='Senador'] # Exemplo de vinculos somados 

,codigo,esfera,sexo,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
0,111105,Estadual,Feminino,Senador,"Membros Superiores Do Poder Público, Dirigente...",3
1,111105,Federal,Masculino,Senador,"Membros Superiores Do Poder Público, Dirigente...",2
2,111105,Municipal,Feminino,Senador,"Membros Superiores Do Poder Público, Dirigente...",52
3,111105,Municipal,Masculino,Senador,"Membros Superiores Do Poder Público, Dirigente...",67


In [46]:
cbo_merge['ano'] = 2003

In [47]:
cbo_merge = cbo_merge.rename(columns={'sexo':'genero'})


In [48]:
cbo_merge = cbo_merge[['ano','codigo','nome_ocupacao','grupo_ocupacao','esfera','genero','quantidade_vinculos']]

In [49]:
cbo_merge.dtypes

ano                     int64
codigo                  int32
nome_ocupacao          object
grupo_ocupacao         object
esfera                 object
genero                 object
quantidade_vinculos     Int64
dtype: object

In [50]:
cbo_merge.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [51]:
cbo_merge['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Profissionais De Ensino',
       'Profissionais De Artes De Ensino Superior',
       'Técnicos De Nível Médio',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores Em Serviços De Reparação E Manutenção'],
      dtype=object)

In [52]:
dataset_ref = client.dataset('perfil_remuneracao')

In [53]:
schema = [bigquery.SchemaField('ano', 'INTEGER', description= 'Ano de referência da observação'),
          bigquery.SchemaField('codigo', 'INTEGER', description= 'Código da Classificação Brasileira de Ocupações (CBO)'),
          bigquery.SchemaField('nome_ocupacao', 'STRING', description= 'Nome da ocupação na CBO'),
          bigquery.SchemaField('grupo_ocupacao', 'STRING', description= 'Agrupamento da ocupação'),
          bigquery.SchemaField('esfera', 'STRING', description= 'Nível da esfera do governo referente da observação'),                    
          bigquery.SchemaField('genero', 'STRING', description= 'Gênero autodeclarado ou não'),                    
          bigquery.SchemaField('quantidade_vinculos', 'INTEGER', description= 'Número total de vinculos observados')
]

In [54]:
table_ref = dataset_ref.table('RAIS_cbo_2003_quantidade_vinculos')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(cbo_merge, table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=9aa86fd8-5b76-45a7-b35a-19ef5bd24990>

Analises

In [3]:
os.chdir('G:\\Drives compartilhados\\República.org\\4. Equipes\\Dados e Comunicação\\DADOS E CONHECIMENTO\\415 - Repositório de Dados\\Atualização Dados novos\\CBO\\Analise CBO')

In [4]:
os.listdir()

['dados comparativos - CBO 2003 2022 .gsheet',
 'total por grupo 2022.xlsx',
 'comparar.xlsx',
 'total por grupo 2003.xlsx']

In [5]:
query = '''SELECT *  FROM `repositoriodedadosgpsp.perfil_remuneracao.RAIS_cbo_2003_quantidade_vinculos`'''

In [6]:
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')
df


Downloading: 100%|██████████|


,ano,codigo,nome_ocupacao,grupo_ocupacao,esfera,genero,quantidade_vinculos
0,2003,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Estadual,Feminino,95
1,2003,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Federal,Feminino,14
2,2003,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Municipal,Feminino,15052
3,2003,231105,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Outros,Feminino,86
4,2003,231110,Professor De Nível Superior Na Educação Infant...,Profissionais De Ensino,Federal,Feminino,5
...,...,...,...,...,...,...,...
8801,2003,142605,Gerente De Pesquisa E Desenvolvimento (P&D),"Membros Superiores Do Poder Público, Dirigente...",Outros,Masculino,84
8802,2003,142705,Gerente De Projetos E Serviços De Manutenção,"Membros Superiores Do Poder Público, Dirigente...",Estadual,Masculino,164
8803,2003,142705,Gerente De Projetos E Serviços De Manutenção,"Membros Superiores Do Poder Público, Dirigente...",Federal,Masculino,2
8804,2003,142705,Gerente De Projetos E Serviços De Manutenção,"Membros Superiores Do Poder Público, Dirigente...",Municipal,Masculino,273


In [7]:
df.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [8]:
df['quantidade_vinculos'].sum()

7331405

In [12]:
df[df['nome_ocupacao']=='Auxiliar De Enfermagem']

,ano,codigo,nome_ocupacao,grupo_ocupacao,esfera,genero,quantidade_vinculos
5117,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Estadual,Feminino,58629
5118,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Federal,Feminino,13328
5119,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Municipal,Feminino,79302
5120,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Outros,Feminino,2461
5453,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Estadual,Masculino,8967
5454,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Federal,Masculino,2325
5455,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Municipal,Masculino,12939
5456,2003,322230,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",Outros,Masculino,495


In [13]:
df_agrupado = df.groupby('grupo_ocupacao')['quantidade_vinculos'].sum().reset_index()
df_agrupado 

,grupo_ocupacao,quantidade_vinculos
0,"Membros Superiores Do Poder Público, Dirigente...",549190
1,Profissionais De Artes De Ensino Superior,5269
2,Profissionais De Ciência De Ensino Superior,297869
3,Profissionais De Ensino,1838584
4,"Profissionais De Medicina, Enfermagem E Serviç...",626571
5,"Trabalhadores Agropecuários, Florestais E Da P...",35687
6,Trabalhadores Da Produção De Bens E Serviços I...,397963
7,Trabalhadores De Serviços Administrativos,1884247
8,"Trabalhadores Dos Serviços, Vendedores Do Comé...",782279
9,Trabalhadores Em Serviços De Reparação E Manut...,85530


In [15]:
total = df['quantidade_vinculos'].sum() 
total

7331405

In [20]:
df_agrupado['proporcao'] = (df_agrupado['quantidade_vinculos'] / total) * 100

In [21]:
df_agrupado

,grupo_ocupacao,quantidade_vinculos,proporcao
0,"Membros Superiores Do Poder Público, Dirigente...",549190,7.490924
1,Profissionais De Artes De Ensino Superior,5269,0.071869
2,Profissionais De Ciência De Ensino Superior,297869,4.062918
3,Profissionais De Ensino,1838584,25.078194
4,"Profissionais De Medicina, Enfermagem E Serviç...",626571,8.546397
5,"Trabalhadores Agropecuários, Florestais E Da P...",35687,0.486769
6,Trabalhadores Da Produção De Bens E Serviços I...,397963,5.428196
7,Trabalhadores De Serviços Administrativos,1884247,25.701035
8,"Trabalhadores Dos Serviços, Vendedores Do Comé...",782279,10.670247
9,Trabalhadores Em Serviços De Reparação E Manut...,85530,1.166625


In [24]:
df_agrupado.sort_values(ascending=False, by='quantidade_vinculos')

,grupo_ocupacao,quantidade_vinculos,proporcao
7,Trabalhadores De Serviços Administrativos,1884247,25.701035
3,Profissionais De Ensino,1838584,25.078194
8,"Trabalhadores Dos Serviços, Vendedores Do Comé...",782279,10.670247
4,"Profissionais De Medicina, Enfermagem E Serviç...",626571,8.546397
11,Técnicos De Nível Médio,562226,7.668735
0,"Membros Superiores Do Poder Público, Dirigente...",549190,7.490924
6,Trabalhadores Da Produção De Bens E Serviços I...,397963,5.428196
2,Profissionais De Ciência De Ensino Superior,297869,4.062918
10,Trabalhadores Nos Serviços De Proteção E Segur...,265990,3.62809
9,Trabalhadores Em Serviços De Reparação E Manut...,85530,1.166625


In [25]:
df_agrupado['grupo_ocupacao'].unique()

array(['Membros Superiores Do Poder Público, Dirigentes De Organizações De Interesse Público E De Empresas, Gerentes',
       'Profissionais De Artes De Ensino Superior',
       'Profissionais De Ciência De Ensino Superior',
       'Profissionais De Ensino',
       'Profissionais De Medicina, Enfermagem E Serviços Da Saúde',
       'Trabalhadores Agropecuários, Florestais E Da Pesca',
       'Trabalhadores Da Produção De Bens E Serviços Industriais',
       'Trabalhadores De Serviços Administrativos',
       'Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados',
       'Trabalhadores Em Serviços De Reparação E Manutenção',
       'Trabalhadores Nos Serviços De Proteção E Segurança',
       'Técnicos De Nível Médio'], dtype=object)

# por profissão


pegar
1. Trabalhadores De Serviços Administrativos com 1.884.247	(25.70%)
2.	Profissionais De Ensino	com 1.838.584	(25.07%)
3.	Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados	782279	(10.67%)



In [27]:
df.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [30]:
agrup = df.groupby(['nome_ocupacao', 'grupo_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index() #top profissões
top10 = agrup.sort_values(ascending=False, by='quantidade_vinculos').head(10)

In [51]:
top10 = top10.rename(columns={'quantidade_vinculos':'vinculo_profissoes'})

In [35]:
top10.to_excel('top10 profissoes por nome e grupo.xlsx', index= False)

In [ ]:
top10_merge = top10.merge(df_agrupado, how='left', on='grupo_ocupacao') #pegando total por grupo
top10_merge

In [ ]:
top10_merge = top10_merge.rename(columns={'proporcao':'proporcao_grupo'})
top10_merge

In [83]:
#proporção da ocupação em relação ao total. 
total_vinculos2003 = 7331405 

In [84]:
top10_merge['proporcao_prof_vinculos_totais2003'] = (top10_merge['vinculo_profissoes']/total_vinculos2003)*100 #em relação ao total

In [85]:
top10_merge['proporcao_profissao'] = (top10_merge['vinculo_profissoes'] / top10_merge['quantidade_vinculos'])*100
#proporção da profissao em relação ao grupo 


In [76]:
top10_merge.to_excel('2003 top 10 profissoes + %.xlsx')

dados top 3
1.  Trabalhadores De Serviços Administrativos 
2. Profissionais De Ensino com 
3. Trabalhadores Dos Serviços, Vendedores Do Comércio Em Lojas E Mercados 

In [61]:
adm = df[df['grupo_ocupacao']=='Trabalhadores De Serviços Administrativos']

In [62]:
adm.columns

Index(['ano', 'codigo', 'nome_ocupacao', 'grupo_ocupacao', 'esfera', 'genero',
       'quantidade_vinculos'],
      dtype='object')

In [ ]:
adm_group = adm.groupby(['nome_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index()
adm_group.sort_values(ascending=False, by= 'quantidade_vinculos')

In [ ]:
adm_group_final = adm_group.sort_values(ascending=False, by= 'quantidade_vinculos').head(10)
adm_group_final

In [65]:
total_vinculos_categoria = adm.['quantidade_vinculos'].sum()

In [66]:
adm_group_final['proporcao_categoria'] = (adm_group_final['quantidade_vinculos'] / total_vinculos_categoria)*100


In [67]:
adm_group_final['proporcao_categoria'] = adm_group_final['proporcao_categoria'].round(2)

In [ ]:
adm_group_final

Profissionais De Ensino 

In [94]:
ensino =df[df['grupo_ocupacao']=='Profissionais De Ensino']

In [ ]:
ensino_group = ensino.groupby(['nome_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index() #top profissões
ensino_group

In [ ]:
ensino_group= ensino_group.sort_values(ascending=False, by='quantidade_vinculos').head(10)
ensino_group

In [99]:
total_vinculos_ensino = ensino['quantidade_vinculos'].sum()

In [100]:
ensino_group['proporcao_categoria'] = (ensino_group['quantidade_vinculos'] / total_vinculos_ensino)*100


In [103]:
ensino_group.to_csv('TOP10 grupo ensino.csv', index=False)

Técnicos De Nível Médio

In [105]:
medio = df[df['grupo_ocupacao']=='Técnicos De Nível Médio']

In [106]:
total_medio = medio['quantidade_vinculos'].sum()

In [108]:
mediogroup = medio.groupby(['nome_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index()


In [ ]:
a = df[(df['codigo']>321999)&(df['codigo']<323000)]
a

In [ ]:
a['quantidade_vinculos'].sum()


In [ ]:
a['quantidade_vinculos'].sum()

In [ ]:
mediogroup.sort_values(ascending=False, by= 'quantidade_vinculos')

# maior qtd de vinculos 2022 por profissao

In [92]:
df_agp = df.groupby(['nome_ocupacao', 'grupo_ocupacao']).agg({'quantidade_vinculos': 'sum'}).reset_index()
df_agp.sort_values(ascending=False, by= 'quantidade_vinculos').head(10)

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
107,Assistente Administrativo,Trabalhadores De Serviços Administrativos,751741
1585,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,616824
142,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,598009
1587,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,345480
499,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",200756
1504,Professor Da Educação De Jovens E Adultos Do ...,Profissionais De Ensino,194755
1872,Trabalhador De Serviços De Limpeza E Conservaç...,"Trabalhadores Dos Serviços, Vendedores Do Comé...",192410
140,Auxiliar De Enfermagem,"Profissionais De Medicina, Enfermagem E Serviç...",178446
1529,Professor De Disciplinas Pedagógicas No Ensino...,Profissionais De Ensino,168077
501,Dirigente Do Serviço Público Municipal,"Membros Superiores Do Poder Público, Dirigente...",163410


In [93]:
df_agp = df_agp.sort_values(ascending=False, by= 'quantidade_vinculos')
df_agp

,nome_ocupacao,grupo_ocupacao,quantidade_vinculos
107,Assistente Administrativo,Trabalhadores De Serviços Administrativos,751741
1585,Professor De Nível Médio No Ensino Fundamental,Profissionais De Ensino,616824
142,Auxiliar De Escritório,Trabalhadores De Serviços Administrativos,598009
1587,Professor De Nível Superior Do Ensino Fundamen...,Profissionais De Ensino,345480
499,Dirigente Do Serviço Público Estadual E Distrital,"Membros Superiores Do Poder Público, Dirigente...",200756
...,...,...,...
649,Estatístico Teórico,Profissionais De Ciência De Ensino Superior,1
532,Eletromecânico De Manutenção De Portas Automát...,Trabalhadores Em Serviços De Reparação E Manut...,1
1159,"Operador De Bobinadeira De Tiras A Quente, No ...",Trabalhadores Da Produção De Bens E Serviços I...,1
396,Decorador De Cerâmica,Trabalhadores Da Produção De Bens E Serviços I...,1


In [94]:
df_agp.to_excel('quantidae vinculo 2003.xlsx', index=False)